## 6.4 BERT for question/answering

In [1]:
from transformers import BertTokenizerFast, BertForQuestionAnswering, pipeline, \
                         DataCollatorWithPadding, TrainingArguments, Trainer, \
                         AutoModelForQuestionAnswering, AutoTokenizer
from datasets import Dataset
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from bs4 import BeautifulSoup
import requests

ModuleNotFoundError: No module named 'datasets'

In [52]:
bert_tokenizer = BertTokenizerFast.from_pretrained('bert-large-uncased', return_token_type_ids=True)

qa_bert = BertForQuestionAnswering.from_pretrained('bert-large-uncased')

loading file https://huggingface.co/bert-large-uncased/resolve/main/vocab.txt from cache at /Users/sinanozdemir/.cache/huggingface/transformers/e12f02d630da91a0982ce6db1ad595231d155a2b725ab106971898276d842ecc.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-large-uncased/resolve/main/tokenizer.json from cache at /Users/sinanozdemir/.cache/huggingface/transformers/475d46024228961ca8770cead39e1079f135fd2441d14cf216727ffac8d41d78.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/bert-large-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-large-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-large-uncased/resolve/main/tokenizer_config.json from cache at /Users/sinanozdemir/.cache/huggingface/transformers/300ecd79785b4602752c0085f8a89c3f0232ef367eda291c79a5600f3778b677.20430bd8e1

In [118]:
qa_df = pd.read_csv('../data/qa.csv')

qa_df.shape

(29989, 5)

In [116]:
qa_df.head()

,question,context,start_positions,end_positions,answer
0,What sare the benifts of the blood brain barrir?,Another approach to brain function is to exami...,56,60,isolated from the bloodstream
1,What is surrounded by cerebrospinal fluid?,Another approach to brain function is to exami...,16,16,brain
2,What does the skull protect?,Another approach to brain function is to exami...,11,11,brain
3,What has been injected into rats to produce pr...,Another approach to brain function is to exami...,153,153,chemicals
4,What can cause issues with how the brain works?,Another approach to brain function is to exami...,93,94,brain damage


In [117]:
qa_df.iloc[0]

question            What sare the benifts of the blood brain barrir?
context            Another approach to brain function is to exami...
start_positions                                                   56
end_positions                                                     60
answer                                 isolated from the bloodstream
Name: 0, dtype: object

In [124]:
# index 56, 57, 58, 59, and 60 including question while encoding
bert_tokenizer.decode(bert_tokenizer.encode(qa_df.iloc[0].question, qa_df.iloc[0].context)[56:61])

'isolated from the bloodstream'

In [56]:
# only grab 4,000 examples
qa_dataset = Dataset.from_pandas(qa_df.sample(4000, random_state=42))

# Dataset has a built in train test split method
qa_dataset = qa_dataset.train_test_split(test_size=0.2)

In [57]:
# standard preprocessing here with truncation on to truncate longer text
def preprocess(data):
    return bert_tokenizer(data['question'], data['context'], truncation=True)

qa_dataset = qa_dataset.map(preprocess, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [58]:
# freeze all but the last 2 encoder layers in BERT to speed up training
for name, param in qa_bert.bert.named_parameters():
    if 'encoder.layer.22' in name:
        break
    param.requires_grad = False  # disable training in BERT

In [59]:
data_collator = DataCollatorWithPadding(tokenizer=bert_tokenizer)

In [20]:
batch_size = 32
epochs = 2

training_args = TrainingArguments(
    output_dir='./qa/results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_dir='./qa/logs',
    save_strategy='epoch',
    logging_steps=10,
    evaluation_strategy='epoch',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=qa_bert,
    args=training_args,
    train_dataset=qa_dataset['train'],
    eval_dataset=qa_dataset['test'],
    data_collator=data_collator
)

# Get initial metrics
trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the evaluation set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: __index_level_0__, context, answer, question.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 32


{'eval_loss': 6.270374774932861,
 'eval_runtime': 1428.4792,
 'eval_samples_per_second': 0.56,
 'eval_steps_per_second': 0.018}

In [21]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: __index_level_0__, context, answer, question.
***** Running training *****
  Num examples = 3200
  Num Epochs = 4
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 400


Epoch,Training Loss,Validation Loss
1,4.314100,4.248008
2,4.009000,4.084617


The following columns in the evaluation set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: __index_level_0__, context, answer, question.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 32
Saving model checkpoint to ./qa/results/checkpoint-100
Configuration saved in ./qa/results/checkpoint-100/config.json
Model weights saved in ./qa/results/checkpoint-100/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `BertForQuestionAnswering.forward` and have been ignored: __index_level_0__, context, answer, question.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 32
Saving model checkpoint to ./qa/results/checkpoint-200
Configuration saved in ./qa/results/checkpoint-200/config.json
Model weights saved in ./qa/results/checkpoint-200/pytorch_model.bin


KeyboardInterrupt: 

In [ ]:
# Q/A models are very large and take a long time to fine-tune

In [22]:
trainer.save_model()

Saving model checkpoint to ./qa/results
Configuration saved in ./qa/results/config.json
Model weights saved in ./qa/results/pytorch_model.bin


In [101]:
pipe = pipeline("question-answering", './qa/results', tokenizer=bert_tokenizer)

loading configuration file ./qa/results/config.json
Model config BertConfig {
  "_name_or_path": "./qa/results",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading configuration file ./qa/results/config.json
Model config BertConfig {
  "_name_or_path": "./qa/results",
  "architectures": [
    "BertForQuestionAnswering"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_che

In [92]:
pipe("Where is Sinan living these days?", "Sinan lives in California but Matt lives in Boston.")

{'score': 0.13000917434692383, 'start': 15, 'end': 25, 'answer': 'California'}